In [ ]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
DATE = "2021-06-02"

# Dataset Download

In [ ]:
os.makedirs(f'data/{DATE}', exist_ok=True)

In [ ]:
# http://datos.salud.gob.ar/dataset/vacunas-contra-covid19-dosis-aplicadas-en-la-republica-argentina
# https://sisa.msal.gov.ar/datos/descargas/covid-19/files/datos_nomivac_covid19.zip
! wget https://sisa.msal.gov.ar/datos/descargas/covid-19/files/datos_nomivac_covid19.zip -P data/{DATE}

In [ ]:
!unzip data/{DATE}/datos_nomivac_covid19.zip -d data/{DATE}

# Load Data

In [ ]:
df = pd.read_csv(f"data/{DATE}/datos_nomivac_covid19.csv")
df.shape

### Preprocesamiento

####  Dosis sin fecha de vacunación

In [ ]:
df[df.fecha_aplicacion == "S.I."]["fecha_aplicacion"].count()

In [ ]:
df = df[df.fecha_aplicacion != "S.I."]

In [ ]:
df = df[df.fecha_aplicacion < DATE]

### Unificacion astrazeneca + covidshield

In [ ]:
df.vacuna = df.vacuna.replace('COVISHIELD', 'AstraZeneca')

### Create dates DataFrame

In [ ]:
dates_df = pd.DataFrame(df.fecha_aplicacion.unique(), columns=["fecha_aplicacion"]).sort_values("fecha_aplicacion").reset_index(drop=True)

# Evolutivo segun fecha de vacunación - total pais

In [ ]:
aux_df = df.fecha_aplicacion.value_counts().sort_index().reset_index()
aux_df.columns = ["fecha_aplicacion", "nb_vac"]
aux_df.shape

In [ ]:
aux_df["7 días"] = aux_df.nb_vac.rolling(window=7).mean()
aux_df["28 días"] = aux_df.nb_vac.rolling(window=28).mean()

In [ ]:
fig = px.bar(aux_df, x="fecha_aplicacion", y="nb_vac", labels={"nb_vac": "Cantidad de dosis aplicadas"})
fig.update_traces(marker_color='#C0C0C0')

fig2 = px.line(aux_df, x="fecha_aplicacion", y=["7 días", "28 días"])
fig2.update_traces(mode='markers+lines')
fig2.data[0].line.color = "#0D6400"
fig2.data[1].line.color = "#AE0000"

fig.add_traces(fig2.data)
fig.update_layout(
    xaxis = go.layout.XAxis(
        tickangle = 45),
    title={
        'text': "Evolución de dosis aplicadas - Total país",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    yaxis = dict({"tickformat":",d"}),
    legend_title="Media móvil",
)
fig.add_annotation(
                text=f'Total dosis aplicadas: {aux_df.nb_vac.sum():,}',
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                y=0.96,
                x=0.07,
                bordercolor='#000066',
                bgcolor='#FFFFFF',
                borderwidth=2)
fig.show()

# Acumulado 1&2 dosis - total pais

In [ ]:
aux_df = df.groupby(["fecha_aplicacion", "orden_dosis"]).grupo_etario.count().reset_index()
aux_df.columns = ["fecha_aplicacion", "orden_dosis", "nb_vac"]

In [ ]:
aux_df = aux_df[aux_df.orden_dosis == 1].merge(aux_df[aux_df.orden_dosis == 2], how="outer", on="fecha_aplicacion")
aux_df.columns = ["fecha_aplicacion", "orden_dosis_1", "nb_vac_1", "orden_dosis_2", "nb_vac_2"]
aux_df.fillna(0, inplace=True)
aux_df.nb_vac_2 = aux_df.nb_vac_2.astype(int)
aux_df.orden_dosis_2 = aux_df.orden_dosis_2.astype(int)

In [ ]:
aux_df["primera"] = aux_df.nb_vac_1.cumsum()
aux_df["segunda"] = aux_df.nb_vac_2.cumsum()

In [ ]:
fig = px.line(aux_df, x="fecha_aplicacion", y=["primera", "segunda"])
fig.update_traces(mode='markers+lines')
fig.data[0].line.color = "#0D6400"
fig.data[1].line.color = "#AE0000"

fig.update_layout(
    xaxis = go.layout.XAxis(
        tickangle = 45),
    title={
        'text': "Acumulado 1&2 dosis - Total País",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    yaxis = dict({"tickformat":",d"}),
    yaxis_title="Cantidad de dosis aplicadas",
    legend_title="Dosis acumuladas"
)
fig.add_annotation(
                text=f'Total 1 dosis aplicadas: {aux_df.nb_vac_1.sum():,}',
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                y=0.96,
                x=0.07,
                bordercolor='#000066',
                bgcolor='#FFFFFF',
                borderwidth=2)
fig.add_annotation(
                text=f'Total 2 dosis aplicadas: {aux_df.nb_vac_2.sum():,}',
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                y=0.86,
                x=0.07,
                bordercolor='#000066',
                bgcolor='#FFFFFF',
                borderwidth=2)

fig.show()

# Tipo de vacuna - Total Pais

In [ ]:
part_df = (df.vacuna.value_counts(normalize=True) * 100).round(2).reset_index()
part_df.columns = ["vacuna", "part_%"]
part_df["part_%"] = part_df["part_%"].apply(lambda x: f"{x} %")
part_df

In [ ]:
aux_df = df.groupby(["vacuna", "orden_dosis"]).grupo_etario.count().reset_index()
aux_df.columns = ["vacuna", "orden_dosis", "nb_vac"]
aux_df.orden_dosis = aux_df.orden_dosis.astype(str)

aux_df = aux_df.merge(part_df, on="vacuna")
aux_df.sort_values("part_%", ascending=False, inplace=True)
aux_df["vacuna"] = aux_df["vacuna"] + " - "+ aux_df["part_%"]
aux_df.orden_dosis = aux_df.orden_dosis.str.replace("1", "primera").str.replace("2", "segunda")
aux_df

In [ ]:
fig = px.bar(aux_df, x="vacuna", y='nb_vac', color="orden_dosis", barmode="group", text='nb_vac')
fig.data[0].marker.color = "#0D6400"
fig.data[1].marker.color = "#AE0000"

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

fig.update_layout(
    title={
        'text': "Dosis aplicadas por tipo de Vacuna - Total país",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    yaxis = dict({"tickformat":",d"}),
    yaxis_title="Cantidad de dosis aplicadas",
    xaxis_title="Tipo de vacuna - Participación del total",
    legend_title="Dosis"
)
fig.show()

# Diferimiento de dosis

In [ ]:
def diferimiento_dosis(data):
    aux_df = data.groupby(["fecha_aplicacion", "orden_dosis"]).grupo_etario.count().reset_index()
    aux_df.columns = ["fecha_aplicacion", "orden_dosis", "nb_vac"]

    aux_df = aux_df[aux_df.orden_dosis == 1].merge(aux_df[aux_df.orden_dosis == 2], how="outer", on="fecha_aplicacion")
    aux_df.columns = ["fecha_aplicacion", "orden_dosis_1", "nb_vac_1", "orden_dosis_2", "nb_vac_2"]
    aux_df.fillna(0, inplace=True)
    aux_df.nb_vac_2 = aux_df.nb_vac_2.astype(int)
    aux_df.orden_dosis_2 = aux_df.orden_dosis_2.astype(int)

    # Agregar fechas sin registro
    aux_df = dates_df.merge(aux_df, on="fecha_aplicacion", how="left")
    aux_df.fillna(0, inplace=True)
    
    aux_df["nb_vac_1_cumsum"] = aux_df.nb_vac_1.cumsum()
    aux_df["nb_vac_2_cumsum"] = aux_df.nb_vac_2.cumsum()

    today = aux_df.fecha_aplicacion.max()
    forc_df = pd.DataFrame(pd.date_range(today, periods=15, freq='d') + pd.DateOffset(1), columns=["fecha_aplicacion"])
    forc_df.fecha_aplicacion = forc_df.fecha_aplicacion.astype(str)

    aux_df = pd.concat([aux_df, forc_df]).reset_index(drop=True)
    
    for i in [60, 90]:
        aux_df[f"nb_vac_1_cumsum_shift_{i}_days"] = aux_df.nb_vac_1_cumsum.shift(i)

    return aux_df, today

In [ ]:
def diff_plot(data, today):
    fig = px.line(data, x="fecha_aplicacion", y=["nb_vac_2_cumsum"] + [f"nb_vac_1_cumsum_shift_{i}_days" for i in [60, 90]])
    fig.update_traces(mode='markers+lines')

    color_l = ["#AE0000", "#003333", "#003300"]
    name_l = ["Segundas", "Primeras hace 60 días", "Primeras hace 90 días"]
    for i, (color, name) in enumerate(zip(color_l, name_l)):    
        fig.data[i].line.color = color
        fig.data[i].name = name

    fig.add_vline(x=today, line_width=3, line_dash="dash", line_color='#C0C0C0')
    fig.add_vrect(x0=today, x1=data.fecha_aplicacion.max(), row="all", col=1,
                  annotation_text=today, annotation_position="top left",
                  fillcolor='#C0C0C0', opacity=0.25, line_width=0)

    fig.update_layout(
        xaxis = go.layout.XAxis(
            tickangle = 45),
        title={
            'text': "Diferimiento de segundas dosis - Todos los tipos - Total país",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        yaxis = dict({"tickformat":",d"}),
        yaxis_title="Cantidad de dosis aplicadas",
        legend_title="Dosis acumuladas",
    )
    return fig

In [ ]:
aux_df, today = diferimiento_dosis(df)
fig = diff_plot(aux_df, today)
fig.show()

### Diferimiento de dosis por tipo

In [ ]:
for vac_tipo in list(df.vacuna.unique()):
    df1 = df[df["vacuna"] == vac_tipo]
    aux_df, today = diferimiento_dosis(df1)
    fig = diff_plot(aux_df, today)
    fig.update_layout(title={
        'text': f"Diferimiento de segundas dosis - {vac_tipo} - Total país"
    })
    fig.show()